In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tqdm import tqdm
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')


In [2]:
data=pd.read_csv('../input/sample/sample-2.csv')

In [3]:
data.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-007676813_00577,Complete,https://transcription.si.edu/transcribe/32203/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32203/image/NMAAHC-007676813_00577.jpg,1,"8 E encl\r\n\r\nI Anne H. Elliott, widow do so...",/32203/transcriptions/NMAAHC-007676813_00577.txt,\N,\N,32203,Oaths/Pardons,NaN,NaN
1,NMAAHC-007676818_00567,Complete,https://transcription.si.edu/transcribe/32796/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32796/image/NMAAHC-007676818_00567.jpg,1,I E C Whaley solemnly swear that I have carefu...,/32796/transcriptions/NMAAHC-007676818_00567.txt,\N,\N,32796,Oaths/Pardons,NaN,NaN
2,NMAAHC-007676815_00488-000001,Complete,https://transcription.si.edu/transcribe/32218/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32218/image/NMAAHC-007676815_00488-000001.jpg,1,43M\r\nI Mrs E Montgomery solemnly Swear that ...,/32218/transcriptions/NMAAHC-007676815_00488-0...,\N,\N,32218,Oaths/Pardons,NaN,NaN
3,NMAAHC-007677400_00375,Complete,https://transcription.si.edu/transcribe/26149/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/26149/image/NMAAHC-007677400_00375.jpg,1,"No. 2324\r\nI do solemnly swear or affirm, in ...",/26149/transcriptions/NMAAHC-007677400_00375.txt,\N,\N,26149,Oaths/Pardons,NaN,NaN
4,NMAAHC-007677378_00480,Complete,https://transcription.si.edu/transcribe/27345/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/27345/image/NMAAHC-007677378_00480.jpg,1,August 24. 1865\r\nAPPLICATION AND CERTIFICATE...,/27345/transcriptions/NMAAHC-007677378_00480.txt,\N,\N,27345,Oaths/Pardons,NaN,NaN


In [4]:
data.shape

(20, 15)

In [5]:
text=data["transcription_text"]

In [6]:
text[1]


'I E C Whaley solemnly swear that I have carefully read the Amnesty\r\nProclamation issued by Andrew Johnson, President of the United State of America, on May 29th, 1865, and that I am not excepted from the benefits of the Proclamation by any one of the fourteen exceptions therein made [[Strikethrough]] except [[strikethrough/]] \r\nE C Whaley\r\nSworn to and subscribed before me at Charleston SC this 28\r\nday of Sept 1865\r\n\r\nLevi Steuben\r\nMaj. 47th P.M.\r\n\r\nUnited States of America\r\nE C Whaley of the city [[strikethrough]] county [[strikethrough/]] Charleston Sate of South Ca do solemnly swear in the presence of the Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves-SO HELP ME GOD\r\n\r\nE C Wha

# using NLTK

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [8]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [9]:
sent = preprocess(text[1])
sent[:10]

[('I', 'PRP'),
 ('E', 'VBP'),
 ('C', 'NNP'),
 ('Whaley', 'NNP'),
 ('solemnly', 'RB'),
 ('swear', 'VBP'),
 ('that', 'IN'),
 ('I', 'PRP'),
 ('have', 'VBP'),
 ('carefully', 'RB')]

**first rule : a noun phrase, NP, should be formed whenever the chunker finds an optional determiner, DT, followed by any number of adjectives, JJ, and then a noun, NN.**

In [10]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

*Chunking*

In [11]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs[:10])

[('I', 'PRP'), ('E', 'VBP'), ('C', 'NNP'), ('Whaley', 'NNP'), ('solemnly', 'RB'), ('swear', 'VBP'), ('that', 'IN'), ('I', 'PRP'), ('have', 'VBP'), ('carefully', 'RB')]


*IOB tags*

In [12]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
iob_tagged = tree2conlltags(cs)
pprint(iob_tagged[:5])

[('I', 'PRP', 'O'),
 ('E', 'VBP', 'O'),
 ('C', 'NNP', 'O'),
 ('Whaley', 'NNP', 'O'),
 ('solemnly', 'RB', 'O')]


> With the function nltk.ne_chunk(), we can recognize named entities using a classifier, the classifier adds category labels such as PERSON, ORGANIZATION, and GPE.
like :PERSON Andrew/NNP Johnson , GPE America , PERSON Whaley

In [13]:
ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(text[1])))
print(ne_tree)

(S
  I/PRP
  E/VBP
  C/NNP
  Whaley/NNP
  solemnly/RB
  swear/VBP
  that/IN
  I/PRP
  have/VBP
  carefully/RB
  read/VBN
  the/DT
  (ORGANIZATION Amnesty/NNP)
  Proclamation/NNP
  issued/VBN
  by/IN
  (PERSON Andrew/NNP Johnson/NNP)
  ,/,
  President/NNP
  of/IN
  the/DT
  (ORGANIZATION United/NNP State/NNP)
  of/IN
  (GPE America/NNP)
  ,/,
  on/IN
  May/NNP
  29th/CD
  ,/,
  1865/CD
  ,/,
  and/CC
  that/IN
  I/PRP
  am/VBP
  not/RB
  excepted/VBN
  from/IN
  the/DT
  benefits/NNS
  of/IN
  the/DT
  Proclamation/NNP
  by/IN
  any/DT
  one/CD
  of/IN
  the/DT
  fourteen/JJ
  exceptions/NNS
  therein/RB
  made/VBD
  [/NNP
  [/NNP
  Strikethrough/NNP
  ]/NNP
  ]/NNP
  except/IN
  [/NNP
  [/NNP
  strikethrough//VBD
  ]/NNP
  ]/NNP
  E/NNP
  C/NNP
  (PERSON Whaley/NNP Sworn/NNP)
  to/TO
  and/CC
  subscribed/VBN
  before/IN
  me/PRP
  at/IN
  (ORGANIZATION Charleston/NNP)
  SC/NNP
  this/DT
  28/CD
  day/NN
  of/IN
  Sept/NNP
  1865/CD
  Levi/NNP
  (PERSON Steuben/NNP)
  Maj./NNP
  47th/C

# using Spacy

> SpaCy’s named entity recognition has been trained on the OntoNotes 5 corpus and it supports the following entity types:

![](https://miro.medium.com/max/1400/1*qQggIPMugLcy-ndJ8X_aAA.png)

In [14]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [15]:
text[1]

'I E C Whaley solemnly swear that I have carefully read the Amnesty\r\nProclamation issued by Andrew Johnson, President of the United State of America, on May 29th, 1865, and that I am not excepted from the benefits of the Proclamation by any one of the fourteen exceptions therein made [[Strikethrough]] except [[strikethrough/]] \r\nE C Whaley\r\nSworn to and subscribed before me at Charleston SC this 28\r\nday of Sept 1865\r\n\r\nLevi Steuben\r\nMaj. 47th P.M.\r\n\r\nUnited States of America\r\nE C Whaley of the city [[strikethrough]] county [[strikethrough/]] Charleston Sate of South Ca do solemnly swear in the presence of the Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves-SO HELP ME GOD\r\n\r\nE C Wha

In [16]:
doc = nlp(text[1])
pprint([(X.text, X.label_) for X in doc.ents])

[('Whaley', 'PERSON'),
 ('Amnesty', 'ORG'),
 ('Andrew Johnson', 'PERSON'),
 ('the United State of America', 'ORG'),
 ('May 29th, 1865', 'DATE'),
 ('Proclamation', 'LOC'),
 ('one', 'CARDINAL'),
 ('fourteen', 'CARDINAL'),
 ('C Whaley', 'ORG'),
 ('Charleston SC', 'ORG'),
 ('this 28\r\nday of', 'DATE'),
 ('Sept 1865', 'DATE'),
 ('Levi Steuben', 'PERSON'),
 ('47th', 'ORDINAL'),
 ('P.M.', 'GPE'),
 ('United States of America', 'GPE'),
 ('Whaley', 'PERSON'),
 ('the United States', 'GPE'),
 ('the Union of the States', 'ORG'),
 ('the Emancipation of Slaves-SO', 'ORG'),
 ('C Whaley', 'ORG'),
 ('Charleston SC', 'ORG'),
 ('this 28\r\nday of', 'DATE'),
 ('Sept 1865', 'DATE'),
 ('Levi Steuben', 'PERSON'),
 ('May 47', 'DATE'),
 ('P.M.', 'NORP'),
 ('5 feet 10\r\ninches', 'MONEY'),
 ('39 years', 'DATE'),
 ('Oaths', 'PERSON'),
 ('the Department of State', 'ORG'),
 ('Levi Steuben', 'PERSON'),
 ('47th', 'ORDINAL'),
 ('City of Charleston', 'ORG')]


In [17]:
pprint([(X, X.ent_iob_, X.ent_type_) for X in doc][:20])

[(I, 'O', ''),
 (E, 'O', ''),
 (C, 'O', ''),
 (Whaley, 'B', 'PERSON'),
 (solemnly, 'O', ''),
 (swear, 'O', ''),
 (that, 'O', ''),
 (I, 'O', ''),
 (have, 'O', ''),
 (carefully, 'O', ''),
 (read, 'O', ''),
 (the, 'O', ''),
 (Amnesty, 'B', 'ORG'),
 (
, 'O', ''),
 (Proclamation, 'O', ''),
 (issued, 'O', ''),
 (by, 'O', ''),
 (Andrew, 'B', 'PERSON'),
 (Johnson, 'I', 'PERSON'),
 (,, 'O', '')]


* the three most frequent tokens 

In [18]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('Levi Steuben', 3), ('Whaley', 2), ('C Whaley', 2)]

In [19]:
sampl = nlp(text[2])
len(sampl.ents)

32

In [20]:
labels = [x.label_ for x in sampl.ents]
Counter(labels)

Counter({'CARDINAL': 3,
         'ORG': 11,
         'GPE': 3,
         'DATE': 8,
         'PERSON': 6,
         'QUANTITY': 1})

we need to optimize more the result !!

In [21]:
text[5]

'I Wm Whaley solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, except\r\n\r\nWilliam Whaley\r\nSworn to and subscribed before me at Charleston SC this 3 day of August 1865.\r\n\r\nLevi Stuber\r\nMay 4 Pa Vols\r\nProv. Marshal \r\nCity of Charleston\r\n\r\n[[left margin]]\r\nDuplicate\r\n[[/left margin]]\r\n\r\nNo. 38 \r\nUnited States of America. \r\n\r\n1 Wm Whaley of the County of Charleston State of So Carolina do solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the State thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation

In [22]:
sampl2 = nlp(text[5])

In [23]:
sentences = [x for x in sampl2.sents]
print(sentences)

[I Wm Whaley solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, except

William Whaley
Sworn to and subscribed before me at Charleston SC this 3 day of August 1865., 

Levi Stuber
May 4 Pa Vols
Prov., Marshal 
City of Charleston

, [[left margin]]
Duplicate
[[/left margin]]

No. 38 
United States of America., 

1 Wm Whaley of the County of Charleston State of So Carolina do solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the State thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation or slaves— SO HELP ME GOD., 

William Whaley

In [24]:
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

> here’s the above sentence and its dependencies how they looks like:

In [25]:
displacy.render(nlp(str(sentences)), style='dep', jupyter = True, options = {'distance': 120})

**some pre-processing and lemmatization**

In [26]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(sentences)) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']][:20]

[('[', 'X', '['),
 ('Wm', 'PROPN', 'Wm'),
 ('Whaley', 'PROPN', 'Whaley'),
 ('solemnly', 'ADV', 'solemnly'),
 ('swear', 'VERB', 'swear'),
 ('carefully', 'ADV', 'carefully'),
 ('read', 'VERB', 'read'),
 ('Amnesty', 'PROPN', 'Amnesty'),
 ('Proclamation', 'PROPN', 'Proclamation'),
 ('issued', 'VERB', 'issue'),
 ('ANDREW', 'PROPN', 'ANDREW'),
 ('JOHNSON', 'PROPN', 'JOHNSON'),
 ('President', 'PROPN', 'President'),
 ('United', 'PROPN', 'United'),
 ('States', 'PROPN', 'States'),
 ('America', 'PROPN', 'America'),
 ('29th', 'NOUN', '29th'),
 ('1865', 'NUM', '1865'),
 ('excepted', 'VERB', 'except'),
 ('benefits', 'NOUN', 'benefit')]

In [27]:
dict([(str(x), x.label_) for x in nlp(str(sentences)).ents])

{'Amnesty Proclamation': 'ORG',
 'ANDREW JOHNSON': 'ORG',
 'the United States of America': 'GPE',
 'May 29th, 1865': 'DATE',
 'one': 'CARDINAL',
 'fourteen': 'CARDINAL',
 'William Whaley': 'PERSON',
 'Charleston': 'GPE',
 'this 3 day of August 1865': 'DATE',
 'Levi Stuber': 'PERSON',
 'May 4': 'DATE',
 'Vols': 'ORG',
 'Prov': 'ORG',
 'Marshal \r\nCity of Charleston\r\n\r\n': 'ORG',
 '38': 'CARDINAL',
 'United States of America': 'GPE',
 'Carolina': 'LOC',
 'the United States': 'GPE',
 'the Union of the State': 'ORG',
 'Emancipation': 'LOC',
 'this 3 day': 'DATE',
 '1865': 'DATE',
 'Levi': 'PERSON',
 '47': 'CARDINAL',
 'Brown': 'PERSON',
 '5 feet 8 inches': 'QUANTITY',
 '48 years': 'DATE',
 'Lawyer': 'ORG',
 'the Department of State': 'ORG'}

In [28]:
print([(x, x.ent_iob_, x.ent_type_) for x in sampl2][:30])

[(I, 'O', ''), (Wm, 'O', ''), (Whaley, 'O', ''), (solemnly, 'O', ''), (swear, 'O', ''), (that, 'O', ''), (I, 'O', ''), (have, 'O', ''), (carefully, 'O', ''), (read, 'O', ''), (the, 'O', ''), (Amnesty, 'B', 'ORG'), (Proclamation, 'I', 'ORG'), (issued, 'O', ''), (by, 'O', ''), (ANDREW, 'B', 'ORG'), (JOHNSON, 'I', 'ORG'), (,, 'O', ''), (President, 'O', ''), (of, 'O', ''), (the, 'B', 'GPE'), (United, 'I', 'GPE'), (States, 'I', 'GPE'), (of, 'I', 'GPE'), (America, 'I', 'GPE'), (,, 'O', ''), (on, 'O', ''), (May, 'B', 'DATE'), (29th, 'I', 'DATE'), (,, 'I', 'DATE')]


In [29]:
displacy.render(nlp(str(sampl)), jupyter=True, style='ent')

> function that could help us in extracting specific entities

In [30]:
def process(choice,text ):
    doc = nlp(text)
    d = []
    for ent in doc.ents:
        d.append((ent.label_, ent.text))
        df = pd.DataFrame(d, columns=('named entity', 'output'))
        ORG_named_entity = df.loc[df['named entity'] == 'ORG']
        PERSON_named_entity = df.loc[df['named entity'] == 'PERSON']
        GPE_named_entity = df.loc[df['named entity'] == 'GPE']
        DATE_named_entity = df.loc[df['named entity'] == 'DATE']
    if choice == 'organization':
        results = ORG_named_entity
        num_of_results = len(results)
    elif choice == 'person':
        results = PERSON_named_entity
        num_of_results = len(results)
    elif choice == 'geopolitical':
        results = GPE_named_entity
        num_of_results = len(results)
    elif choice == 'date':
            results = DATE_named_entity
            num_of_results = len(results)
            

#         return displacy.render(nlp(str(sampl)), jupyter=True, style='ent')
    return (results, num_of_results )

In [31]:
process('date', text[1])

(   named entity             output
 4          DATE     May 29th, 1865
 10         DATE  this 28\r\nday of
 11         DATE          Sept 1865
 22         DATE  this 28\r\nday of
 23         DATE          Sept 1865
 25         DATE             May 47
 28         DATE           39 years,
 7)

In [32]:
for i in range(len(text)):
    print("text num", i)
    print(process('person', text[i]))

text num 0
(   named entity           output
0        PERSON  Anne H. Elliott
2        PERSON   Andrew Johnson
8        PERSON  S. Wilde Durson
11       PERSON   Anne H Elliott
13       PERSON   Anne H Elliott
24       PERSON         S. Wilde
26       PERSON            Hazel
29       PERSON  S. Wilde Durson, 8)
text num 1
(   named entity          output
0        PERSON          Whaley
2        PERSON  Andrew Johnson
12       PERSON    Levi Steuben
16       PERSON          Whaley
24       PERSON    Levi Steuben
29       PERSON           Oaths
31       PERSON    Levi Steuben, 7)
text num 2
(   named entity             output
7        PERSON                Sgd
12       PERSON        Levi Stuber
16       PERSON  Mrs E. Montgomery
19       PERSON                Sgd
24       PERSON        Levi Stuber
30       PERSON              Oaths, 6)
text num 3
(  named entity      output
5       PERSON          SO
6       PERSON  E.B. Towne
9       PERSON  Hu Kennedy, 3)
text num 4
(  named entity    

In [33]:
for i in range(len(text)):
    print("text num", i)
    print(process('date', text[i]))

text num 0
(   named entity         output
4          DATE  May 29th 1865
7          DATE           1865
22         DATE           1865
28         DATE       63 years, 4)
text num 1
(   named entity             output
4          DATE     May 29th, 1865
10         DATE  this 28\r\nday of
11         DATE          Sept 1865
22         DATE  this 28\r\nday of
23         DATE          Sept 1865
25         DATE             May 47
28         DATE           39 years, 7)
text num 2
(   named entity          output
4          DATE  May 29th, 1865
10         DATE     this 31 day
11         DATE            1865
13         DATE        47th Pa.
22         DATE     this 31 day
23         DATE            1865
25         DATE        47th Pa.
29         DATE        61 years, 8)
text num 3
(  named entity                             output
0         DATE                               2324
8         DATE  this 31st day of August A.D. 1865, 2)
text num 4
(  named entity      output
0         DATE  August 2